# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (interface problem)

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D, using a domain decomposition approach for $\Omega = \Omega_1 \cup \Omega_2$, and introducing a lagrange multiplier to handle the continuity of the solution across
the interface $\Gamma$ between $\Omega_1$ and $\Omega_2$.

The resulting weak formulation is:
$$
\text{find }u_1 \in V(\Omega_1), u_2 \in V(\Omega_2), \eta \in E(\Gamma)
$$
s.t.
$$
\int_{\Omega_1} \nabla u_1 \cdot \nabla v_1 dx +
\int_{\Omega_2} \nabla u_2 \cdot \nabla v_2 dx +
\int_{\Gamma} \lambda (v_1 - v_2) ds = 0,
\qquad \forall v_1 \in V(\Omega_1), v_2 \in V(\Omega_2)
$$
and
$$
\int_{\Gamma} \eta  (u_1 - u_2) ds = 0,
\qquad \forall \eta \in E(\Gamma)
$$
where boundary conditions on $\partial\Omega$ are embedded in $V(\Omega_i) \subset H^1(\Omega_i)$, $i = 1, 2$, and $E(\Gamma) \subset L^2(\Gamma)$.

This example is a prototypical case of problems containing interface restricted variables (the Lagrange multiplier, in this case).

In [ ]:
from numpy import isclose, sqrt, where
from petsc4py import PETSc
from ufl import grad, inner, Measure, TestFunction, TrialFunction
from dolfinx import *
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import assemble_scalar, locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot
from multiphenics import *

### Mesh

In [ ]:
if MPI.size(MPI.comm_world) > 1:
    mesh_ghost_mode = GhostMode.shared_facet # shared_facet ghost mode is required by dS
else:
    mesh_ghost_mode = GhostMode.none
with XDMFFile(MPI.comm_world, "data/circle.xdmf") as infile:
    mesh = infile.read_mesh(GhostMode.none)
with XDMFFile(MPI.comm_world, "data/circle_subdomains.xdmf") as infile:
    subdomains = infile.read_mf_size_t(mesh)
with XDMFFile(MPI.comm_world, "data/circle_boundaries.xdmf") as infile:
    boundaries = infile.read_mf_size_t(mesh)
cells_Omega_1 = where(subdomains.values == 1)[0]
cells_Omega_2 = where(subdomains.values == 2)[0]
facets_partial_Omega = where(boundaries.values == 1)[0]
facets_Gamma = where(boundaries.values == 2)[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)
dS = Measure("dS")(subdomain_data=boundaries)
dS = dS(2) # restrict to the interface, which has facet ID equal to 2

### With domain decomposition

In [ ]:
# Define a block function space
V = FunctionSpace(mesh, ("Lagrange", 2))
dofs_V_Omega_1 = locate_dofs_topological(V, subdomains.dim, cells_Omega_1)
dofs_V_Omega_2 = locate_dofs_topological(V, subdomains.dim, cells_Omega_2)
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
W = BlockFunctionSpace([V, V, V], restrict=[dofs_V_Omega_1, dofs_V_Omega_2, dofs_V_Gamma])

In [ ]:
# Define trial and test functions
u1u2l = BlockTrialFunction(W)
(u1, u2, l) = block_split(u1u2l)
v1v2m = BlockTestFunction(W)
(v1, v2, m) = block_split(v1v2m)

In [ ]:
# Define problem block forms
a = [[inner(grad(u1), grad(v1))*dx(1), 0                              ,   l("-")*v1("-")*dS ],
     [0                              , inner(grad(u2), grad(v2))*dx(2), - l("+")*v2("+")*dS ],
     [m("-")*u1("-")*dS              , - m("+")*u2("+")*dS            , 0                   ]]
f =  [v1*dx(1)                       , v2*dx(2)                       , 0                   ]
A = BlockForm2(a, [W, W])
F = BlockForm1(f, [W])

In [ ]:
# Define boundary conditions
zero = Function(V)
dofs_W0_partial_Omega = locate_dofs_topological((W.sub(0), V), boundaries.dim, facets_partial_Omega)
dofs_W1_partial_Omega = locate_dofs_topological((W.sub(1), V), boundaries.dim, facets_partial_Omega)
bc1 = DirichletBC(zero, dofs_W0_partial_Omega, W.sub(0))
bc2 = DirichletBC(zero, dofs_W1_partial_Omega, W.sub(1))
bcs = BlockDirichletBC([[bc1], [bc2], []], W)

In [ ]:
# Solve
u1u2l = BlockFunction(W)
solver_parameters = {"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"}
block_solve(A, u1u2l, F, bcs, petsc_options=solver_parameters)

In [ ]:
plot(u1u2l[0])

In [ ]:
plot(u1u2l[1])

### Without domain decomposition

In [ ]:
# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)

In [ ]:
# Define problem forms
a_ex = inner(grad(u), grad(v))*dx
f_ex = v*dx

In [ ]:
# Define Dirichlet BC object on Gamma
dofs_V_partial_Omega = locate_dofs_topological(V, boundaries.dim, facets_partial_Omega)
bc_ex = DirichletBC(zero, dofs_V_partial_Omega)

In [ ]:
# Solve
u_ex = Function(V)
solve(a_ex == f_ex, u_ex, bc_ex, petsc_options=solver_parameters)
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
plot(u_ex)

### Comparison and error compuation

In [ ]:
u_ex1_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(u_ex, u_ex)*dx(1))))
u_ex2_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(u_ex, u_ex)*dx(2))))
err1_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(u_ex - u1u2l[0], u_ex - u1u2l[0])*dx(1))))
err2_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(u_ex - u1u2l[1], u_ex - u1u2l[1])*dx(2))))
print("Relative error on subdomain 1", err1_norm/u_ex1_norm)
print("Relative error on subdomain 2", err2_norm/u_ex2_norm)
assert isclose(err1_norm/u_ex1_norm, 0., atol=1.e-10)
assert isclose(err2_norm/u_ex2_norm, 0., atol=1.e-10)